This tutorial will introduce you to the 'simpleserial' communications system. It will show you how to perform different operations on data based on input from the ChipWhisperer software. This can be used for building your own system which you wish to 'break'.

## What is SimpleSerial
SimpleSerial is the communications protocol used for almost all of the ChipWhisperer demo project. It's a very basic serial protocol which can be easily implemented on most systems. This system communicates using a standard asyncronous serial protocol, 38400 baud, 8-N-1.

All messages are sent in ASCII-text, and are normally terminated with a line-feed ('\n'). This allows you to interact with the simpleserial system over a standard terminal emulator.

`x`
>Sending a 'x' resets the buffers. This does not require a line-feed termination. It is suggested to always send a stream of x's to initilize the system in case the device was already in some other mode due to noise/corruption.

`k00112233445566778899AABBCCDDEEFF\n`
>Loads the encryption key `00112233445566778899AABBCCDDEEFF` into the system. If not called the system may use some default key.

`pAABBCCDDEEFF00112233445566778899\n`
>Encrypts the data `AABBCCDDEEFF00112233445566778899` with the key loaded with the 'k' command. The system will respond with a string starting with r, as shown next.

`rCBBD4A2B34F2571758FF6A797E09859D\n`
>This is the response from the system. If data has been encrypted with a 'p' for example, the system will respond with the 'r' sequence automatically. So sending the earlier example means the result of the encryption was `cbbd4a2b34f2571758ff6a797e09859d`.

## Building the Basic Example
To bulid the basic example, you'll need an appropriate compiler for your target. For the ChipWhisperer Lite/Xmega platform, you'll need avr-gcc and avr-libc, while if you're using an ARM target (like the ChipWhisperer Lite/STM32 platform), your need the gnu toolchain for ARM devices. If you're unsure, you can run the block below:

In [1]:
%%bash
#check for avr-gcc
avr-gcc --version

#check for ARM gcc
arm-none-eabi-gcc --version

avr-gcc.exe (WinAVR 20100110) 4.3.3
Copyright (C) 2008 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.

arm-none-eabi-gcc.exe (GNU Tools for Arm Embedded Processors 7-2017-q4-major) 7.2.1 20170904 (release) [ARM/embedded-7-branch revision 255204]
Copyright (C) 2017 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.

